# Общие указания

Аналогично ДЗ 1

каждое задание - 2 бала, доп задания - 1 бал. Всего 9 - потом приводится к 10.

# Задание 1.

Реализуйте непараметрический LDA (лекция 2, слайд 34). Возьмите датасеты из предыдущего ДЗ (3 задание) (если не делали - подберите или сгенерируйте) и попытайтесь побить затюненный регулеризованный LDA (если не делали предыдущее задание или ваша реализация получилась неудачной, то можете взять реализацию из sklearn) подбирая kernel (перебирайте популярные, а также попробуйте придумать свой kernel), lambda (можно подбирать константу, а можно - функцию, лекция 2 - слайд 26). Сравните время работы алгоритмов.

**Дополнительно**: реализуйте также local likelihood logistic regression (слайды 27 и 33 из второй лекции в помощь) и сравните с моделями из основной части.

In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

In [2]:
class RegularizedLDA(BaseEstimator, ClassifierMixin):
    def __init__(self, alpha=0, beta=0):
        self.alpha = alpha
        self.beta = beta

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        n_features = X.shape[1]
        
        pi_k = []
        mu_k = []
        cov_k = []
        for c in self.classes_:
            X_c = X[y == c]
            pi_k.append(len(X_c) / len(X))
            mu_k.append(X_c.mean(axis=0))
            cov_k.append(np.cov(X_c, rowvar=False))
        self.pi_k = np.array(pi_k)
        self.mu_k = np.array(mu_k)
        self.cov_k = np.array(cov_k)

        
        sigma = np.zeros((n_features, n_features))
        for k in range(n_classes):
            sigma += np.sum(y == self.classes_[k]) * self.cov_k[k]
        sigma /= (X.shape[0] - n_classes)
        
        diag_sigma = np.diag(np.diag(sigma))  
        sigma = self.beta * sigma + (1 -self.beta) * diag_sigma

        sigma_regularized = np.zeros_like(self.cov_k)
        for k in range(n_classes):
            sigma_regularized[k] = self.alpha * self.cov_k[k] + (1 - self.alpha) * sigma
        self.sigma = sigma_regularized
        
        return self

    def predict(self, X):    
        scores = []
        for k in range(len(self.classes_)):
            inv_sigma_k = np.linalg.inv(self.sigma[k])
            delta = X @ inv_sigma_k @ self.mu_k[k] - 0.5 * self.mu_k[k].T @ inv_sigma_k @ self.mu_k[k] + np.log(self.pi_k[k])
            scores.append(delta)
        scores = np.array(scores).T
        return self.classes_[np.argmax(scores, axis=1)]

In [3]:
class NonparametricLDA(BaseEstimator, ClassifierMixin):
    def __init__(self, lambda_=1, kernel='gaussian'):
        self.lambda_ = lambda_
        self.kernel = kernel
        self.kernel_name_to_func = {
            'gaussian': self.gaussian_kernel,
            'epanechnikov': self.epanechnikov_kernel,
            'tricube': self.tricube_kernel,
            'test': self.test_kernel
        }

    def gaussian_kernel(self, distances, lambda_):
        p_ = distances.shape[1]
        return np.exp(-0.5 * (distances / lambda_)**2) / (np.sqrt(2 * np.pi) * lambda_)**(p_ / 2)

    def epanechnikov_kernel(self, distances, lambda_):
        norm_distances = distances / lambda_
        mask = norm_distances <= 1
        kernel_values = np.zeros_like(norm_distances)
        kernel_values[mask] = 0.75 * (1 - norm_distances[mask]**2) / lambda_
        return kernel_values

    def tricube_kernel(self, distances, lambda_):
        norm_distances = distances / lambda_
        mask = norm_distances <= 1
        kernel_values = np.zeros_like(norm_distances)
        kernel_values[mask] = (1 - norm_distances[mask]**3)**3
        return kernel_values / 0.874 # нормализуем чтобы использовать как плотность

    def test_kernel(self, distances, lambda_):
        return (self.tricube_kernel(distances, lambda_) + self.gaussian_kernel(distances, lambda_)) / 2

    def kernel_density_estimation(self, X, data, kernel_func, lambda_):
        distances = np.sqrt(((X[:, np.newaxis] - data)**2).sum(axis=2))
        kernel_values = kernel_func(distances, lambda_)
        return np.sum(kernel_values, axis=1) / len(data)

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        self.pi_k = np.array([np.sum(y == cls) / len(y) for cls in self.classes_])
        self.class_data = [X[y == cls] for cls in self.classes_]

    def predict(self, X):
        n_samples = X.shape[0]
        n_classes = len(self.classes_)
        kernel_func = self.kernel_name_to_func[self.kernel]
        
        density_matrix = np.zeros((n_samples, n_classes))
        for i, cls_data in enumerate(self.class_data):
            density_matrix[:, i] = self.kernel_density_estimation(X, cls_data, kernel_func, self.lambda_)
        
        class_probs = self.pi_k * density_matrix
        norm_factors = np.sum(class_probs, axis=1, keepdims=True)

        probs = class_probs / norm_factors

        return self.classes_[np.argmax(probs, axis=1)]


In [4]:
import warnings
warnings.filterwarnings("ignore")

dataset_names = ["iris", "wine", "soybean", "vehicle", "tic-tac-toe", "banknote-authentication", "spambase"]

datasets = []
from sklearn.datasets import make_classification

for name in dataset_names:
    data = fetch_openml(name, as_frame=True, parser="pandas")
    X = data.data
    y = data.target.astype("category").cat.codes
    datasets.append((name, X, y))


lda_reg_params = {'alpha': np.linspace(0, 1, 10), 'beta': np.linspace(0, 1, 10)}
lda_np_params = {'lambda_': np.linspace(0.01, 2, 20), 'kernel': ['gaussian', 'epanechnikov', 'tricube', 'test']}


best_models = {}

for name, X, y in datasets:
    
    num_cols = X.select_dtypes(include=['int64', 'float64']).columns
    cat_cols = X.select_dtypes(include=['object', 'category']).columns

    for col in cat_cols:
         X[col] = LabelEncoder().fit_transform(X[col])
    
    if y.dtype == 'object':
        y = LabelEncoder().fit_transform(y)
    else:
        y = y.astype(int)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=424, stratify=y)

    
    lda_reg = RegularizedLDA()
    lda_reg_gs = GridSearchCV(lda_reg, param_grid=lda_reg_params, cv=StratifiedKFold(5), scoring="accuracy")
    lda_reg_gs.fit(X_train, y_train)

    start_reg_fit = time.time()
    RegularizedLDA().fit(X_train, y_train)
    elapsed_reg_time_fit = time.time() - start_reg_fit
    
    lda_reg_best = lda_reg_gs.best_estimator_

    start_reg = time.time()
    lda_reg_pred = lda_reg_best.predict(X_test)
    elapsed_reg_time = time.time() - start_reg

    lda_reg_acc = accuracy_score(y_test, lda_reg_pred)

    lda_np = NonparametricLDA()
    lda_np_gs = GridSearchCV(lda_np, param_grid=lda_np_params, cv=StratifiedKFold(5), scoring="accuracy")
    lda_np_gs.fit(X_train, y_train)
    start_np_fit = time.time()
    NonparametricLDA().fit(X_train, y_train)
    elapsed_np_time_fit = time.time() - start_np_fit

    lda_np_best = lda_np_gs.best_estimator_
    
    start_np = time.time()
    lda_np_pred = lda_np_best.predict(X_test)
    elapsed_np_time = time.time() - start_np

    lda_np_acc = accuracy_score(y_test, lda_np_pred)

    best_models[name] = {'lda_reg_best_params': lda_reg_gs.best_params_, 'lda_np_best_params': lda_np_gs.best_params_}
    
    print(f"Dataset: '{name}' with {len(y)} objects")
    print(f"RegularizedLDA time to fit: {elapsed_reg_time_fit}, and NonparametricLDA time to fit: {elapsed_np_time_fit}")
    print(f"RegularizedLDA time to predict: {elapsed_reg_time}, and NonparametricLDA time to predict: {elapsed_np_time}")
    print(f"RegularizedLDA best accuracy: {lda_reg_acc}, with params: {lda_reg_gs.best_params_}")
    print(f"NonparametricLDA best accuracy: {lda_np_acc}, with params: {lda_np_gs.best_params_}", end = '\n\n')

Dataset: 'iris' with 150 objects
RegularizedLDA time to fit: 0.0, and NonparametricLDA time to fit: 0.0009930133819580078
RegularizedLDA time to predict: 0.0009975433349609375, and NonparametricLDA time to predict: 0.0
RegularizedLDA best accuracy: 0.9777777777777777, with params: {'alpha': 0.0, 'beta': 0.7777777777777777}
NonparametricLDA best accuracy: 0.9555555555555556, with params: {'kernel': 'gaussian', 'lambda_': 0.11473684210526315}

Dataset: 'wine' with 178 objects
RegularizedLDA time to fit: 0.001056671142578125, and NonparametricLDA time to fit: 0.001028299331665039
RegularizedLDA time to predict: 0.0, and NonparametricLDA time to predict: 0.0
RegularizedLDA best accuracy: 0.9814814814814815, with params: {'alpha': 0.0, 'beta': 0.8888888888888888}
NonparametricLDA best accuracy: 0.9444444444444444, with params: {'kernel': 'gaussian', 'lambda_': 0.11473684210526315}

Dataset: 'soybean' with 683 objects
RegularizedLDA time to fit: 0.005000114440917969, and NonparametricLDA tim

Получилось побить регуляризованный LDA на некоторых датасетах, причем в одном случае очень значительно (0.23 против 0.90). По поводу времени работы можно увидеть, что предсказание у непараметрического LDA занимает очень значительное время, которое сильно зависит от размера датасета.

#### Доп

In [5]:
import numpy as np
from scipy.optimize import minimize
from sklearn.base import BaseEstimator, ClassifierMixin

class LocalLogisticRegression(BaseEstimator, ClassifierMixin):
    def __init__(self, lambda_=1, kernel='gaussian', max_iter=100):
        self.lambda_ = lambda_
        self.kernel = kernel
        self.max_iter = max_iter
        self.kernel_name_to_func = {
            'gaussian': self.gaussian_kernel,
            'epanechnikov': self.epanechnikov_kernel,
            'tricube': self.tricube_kernel,
            'test': self.test_kernel
        }

    def gaussian_kernel(self, distances, lambda_):
        return np.exp(-0.5 * (distances / lambda_)**2)

    def epanechnikov_kernel(self, distances, lambda_):
        norm_distances = distances / lambda_
        mask = norm_distances <= 1
        kernel_values = np.zeros_like(norm_distances)
        kernel_values[mask] = 0.75 * (1 - norm_distances[mask]**2)
        return kernel_values

    def tricube_kernel(self, distances, lambda_):
        norm_distances = distances / lambda_
        mask = norm_distances <= 1
        kernel_values = np.zeros_like(norm_distances)
        kernel_values[mask] = (1 - norm_distances[mask]**3)**3
        return kernel_values

    def test_kernel(self, distances, lambda_):
        return (self.tricube_kernel(distances, lambda_) + self.gaussian_kernel(distances, lambda_)) / 2

    def kernel_weights(self, X_train, X_test):
        distances = np.sqrt(((X_test[:, np.newaxis, :] - X_train[np.newaxis, :, :])**2).sum(axis=2))
        kernel_func = self.kernel_name_to_func[self.kernel]
        return kernel_func(distances, self.lambda_)

    def _softmax(self, logits):
        exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
        return exp_logits / exp_logits.sum(axis=1, keepdims=True)

    def _log_likelihood(self, params, X, y, kernel_weights, num_classes):
        n_samples, n_features = X.shape
        intercepts = params[:num_classes]
        coefs = params[num_classes:].reshape(num_classes, n_features)
        logits = intercepts + X @ coefs.T
        probs = self._softmax(logits)
        ll = np.sum(kernel_weights * np.log(probs[np.arange(n_samples), y]))
        return -ll
        
    def fit(self, X, y):
        self.X_ = np.asarray(X)
        self.y_ = np.asarray(y)
        self.classes_ = np.unique(y)
        self.num_classes_ = len(self.classes_)
        self.n_features_ = X.shape[1]
        return self

    def _global_fit(self, kernel_weights):
        initial_params = np.zeros(self.num_classes_ + self.num_classes_ * self.n_features_)
        result = minimize(
            self._log_likelihood,
            initial_params,
            args=(self.X_, self.y_, kernel_weights, self.num_classes_),
            method='L-BFGS-B',
            options={'maxiter': self.max_iter}
        )
        params = result.x
        intercepts = params[:self.num_classes_]
        coefs = params[self.num_classes_:].reshape(self.num_classes_, self.n_features_)
        return intercepts, coefs

    def predict_proba(self, X):
        X = np.asarray(X)
        kernel_weights = self.kernel_weights(self.X_, X)
        intercepts, coefs = self._global_fit(kernel_weights)
        logits = intercepts + X @ coefs.T
        return self._softmax(logits)

    def predict(self, X):
        probas = self.predict_proba(X)
        return self.classes_[np.argmax(probas, axis=1)]


In [6]:
import warnings
warnings.filterwarnings("ignore")

dataset_names = ["iris", "wine", "soybean", "vehicle", "tic-tac-toe", "banknote-authentication"]

datasets = []
from sklearn.datasets import make_classification

for name in dataset_names:
    data = fetch_openml(name, as_frame=True, parser="pandas")
    X = data.data
    y = data.target.astype("category").cat.codes
    datasets.append((name, X, y))

llr_params = {'lambda_': np.linspace(0.01, 2, 10), 'kernel': ['gaussian', 'epanechnikov', 'tricube']}

best_models = {}

for name, X, y in datasets:
    
    num_cols = X.select_dtypes(include=['int64', 'float64']).columns
    cat_cols = X.select_dtypes(include=['object', 'category']).columns

    for col in cat_cols:
         X[col] = LabelEncoder().fit_transform(X[col])
    
    if y.dtype == 'object':
        y = LabelEncoder().fit_transform(y)
    else:
        y = y.astype(int)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=424, stratify=y)

    llr = LocalLogisticRegression()
    llr_gs = GridSearchCV(llr, param_grid=llr_params, cv=StratifiedKFold(5), scoring="accuracy")
    llr_gs.fit(X_train, y_train)
    llr_fit = time.time()
    LocalLogisticRegression().fit(X_train, y_train)
    elapsed_llr_time_fit = time.time() - llr_fit

    llr_best = llr_gs.best_estimator_
    
    start_llr = time.time()
    llr_pred = llr_best.predict(X_test)
    elapsed_llr_time = time.time() - start_llr

    llr_acc = accuracy_score(y_test, llr_pred)

    best_models[name] = {'lda_np_best_params': llr_gs.best_params_}
    
    print(f"Dataset: '{name}' with {len(y)} objects")
    print(f"LocalLogisticRegression time to fit: {elapsed_llr_time_fit}")
    print(f"LocalLogisticRegression time to predict: {elapsed_llr_time}")
    print(f"LocalLogisticRegression best accuracy: {llr_acc}, with params: {llr_gs.best_params_}", end = '\n\n')

Dataset: 'iris' with 150 objects
LocalLogisticRegression time to fit: 0.0
LocalLogisticRegression time to predict: 0.04315900802612305
LocalLogisticRegression best accuracy: 0.9555555555555556, with params: {'kernel': 'gaussian', 'lambda_': 0.23111111111111113}

Dataset: 'wine' with 178 objects
LocalLogisticRegression time to fit: 0.0
LocalLogisticRegression time to predict: 0.09591817855834961
LocalLogisticRegression best accuracy: 0.9629629629629629, with params: {'kernel': 'gaussian', 'lambda_': 1.557777777777778}

Dataset: 'soybean' with 683 objects
LocalLogisticRegression time to fit: 0.0
LocalLogisticRegression time to predict: 6.011977434158325
LocalLogisticRegression best accuracy: 0.9170731707317074, with params: {'kernel': 'gaussian', 'lambda_': 2.0}

Dataset: 'vehicle' with 846 objects
LocalLogisticRegression time to fit: 0.0
LocalLogisticRegression time to predict: 6.543075799942017
LocalLogisticRegression best accuracy: 0.7677165354330708, with params: {'kernel': 'gaussian

LocalLogisticRegression  получилось побить оба LDA метода только на одном датасете, однако оно работает значительно дольше даже параметрического LDA

# Задание 2.

Используйте kernel trick для классификации графов с помощью ridge регрессии (она используется для регрессии, но здесь мы сделаем вид, что все ок и будем обрубать значения меньше 0 и больше 1) и svm (в Sklearn можно использовать кастомные kernels).

Датасеты отсюда: https://github.com/FilippoMB/Benchmark_dataset_for_graph_classification

Ядра можно взять отсюда: https://github.com/ysig/GraKeL

Подберите лучшее ядро для всех случаев.

**Дополнительно**: поэксперементируйте с kernel construction на основе kernels из библиотеки и попробуйте предложить свой kernel, который работает лучше

In [7]:
from time import time
import numpy as np
import networkx as nx
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.kernel_ridge import KernelRidge
from grakel import datasets, GraphKernel, graph_from_networkx

In [9]:
kernel_names = [
    "shortest_path", 
    "graphlet_sampling", 
    "pyramid_match", 
    "svm_theta",
    "neighborhood_hash",
    "subtree_wl",
    "odd_sth",
    "propagation",
    "vertex_histogram",
    "weisfeiler_lehman",
    "core_framework"
]

def ridge_clf(K_train, y_train, K_test):
    model = KernelRidge(alpha=1.0, kernel='precomputed')
    model.fit(K_train, y_train)
    y_pred = model.predict(K_test)
    return np.clip(np.round(y_pred), 0, 1).astype(int)

# graph_dataset_names = ["easy_small", "easy", "hard_small", "hard"]
graph_dataset_names = ["hard_small"]

In [10]:
for dataset_name in graph_dataset_names:
    
    print("--------------------------------")
    print("dataset: ", dataset_name)
    print("--------------------------------")
    
    loaded = np.load(f'datasets/{dataset_name}.npz', allow_pickle=True)
    A_train = list(loaded['tr_adj'])
    X_train = loaded['tr_feat']
    y_train = np.argmax(loaded['tr_class'], axis=-1)
    A_test = list(loaded['te_adj'])
    X_test = loaded['te_feat']
    y_test = np.argmax(loaded['te_class'], axis=-1)

    X_train = np.nan_to_num(X_train)
    X_test = np.nan_to_num(X_test)
    
    G_tr, G_te = [], []
    for a, x in zip(A_train, X_train):
        G = nx.from_scipy_sparse_array(a)
        nx.set_node_attributes(G, dict(enumerate(map(tuple, x))), 'features')
        G_tr.append(G)
    for a, x in zip(A_test, X_test):
        G = nx.from_scipy_sparse_array(a)
        nx.set_node_attributes(G, dict(enumerate(map(tuple, x))), 'features')
        G_te.append(G)
    
    G_train = [g for g in graph_from_networkx(G_tr, node_labels_tag='features')]
    G_test = [g for g in graph_from_networkx(G_te, node_labels_tag='features')]
    
    
    
    for k_ in kernel_names:
        start = time()
    
        gk = GraphKernel(kernel=[{"name": k_}], normalize=True)

        K_train = np.nan_to_num(gk.fit_transform(G_train))
        K_test = np.nan_to_num(gk.transform(G_test))
        
        svm_clf = svm.SVC(kernel='precomputed', C=1)
        svm_clf.fit(K_train, y_train)
        svm_y_pred = svm_clf.predict(K_test)
        svm_acc = accuracy_score(y_test, svm_y_pred)
    
        ridge_y_pred = ridge_clf(K_train, y_train, K_test)
        ridge_acc = accuracy_score(y_test, ridge_y_pred)
    
        end = time()
        print(f"{k_} -- SVM Accuracy: {round(svm_acc*100, 2)}% | Ridge Accuracy: {round(ridge_acc*100, 2)}% | Time: {round(end - start, 2)}s")

    print("--------------------------------\n\n\n")

--------------------------------
dataset:  hard_small
--------------------------------
shortest_path -- SVM Accuracy: 69.23% | Ridge Accuracy: 42.31% | Time: 2.78s
graphlet_sampling -- SVM Accuracy: 38.46% | Ridge Accuracy: 34.62% | Time: 31.61s
pyramid_match -- SVM Accuracy: 23.08% | Ridge Accuracy: 34.62% | Time: 2.19s
svm_theta -- SVM Accuracy: 15.38% | Ridge Accuracy: 50.0% | Time: 0.68s
neighborhood_hash -- SVM Accuracy: 61.54% | Ridge Accuracy: 38.46% | Time: 2.12s
subtree_wl -- SVM Accuracy: 15.38% | Ridge Accuracy: 34.62% | Time: 0.01s
odd_sth -- SVM Accuracy: 42.31% | Ridge Accuracy: 34.62% | Time: 14.29s
propagation -- SVM Accuracy: 53.85% | Ridge Accuracy: 34.62% | Time: 1.64s
vertex_histogram -- SVM Accuracy: 15.38% | Ridge Accuracy: 34.62% | Time: 0.01s
weisfeiler_lehman -- SVM Accuracy: 23.08% | Ridge Accuracy: 34.62% | Time: 0.37s
core_framework -- SVM Accuracy: 69.23% | Ridge Accuracy: 42.31% | Time: 13.15s
--------------------------------





Видим что лучшие ядра -- это core_framework и shortest_path

#### Доп

In [11]:
from grakel import Kernel

class ScaledKernel(Kernel):
    def __init__(self, base_kernel, scale=1.0):
        self.base_kernel = base_kernel
        self.scale = scale

    def fit_transform(self, X):
        K = self.base_kernel.fit_transform(X)
        return self.scale * K

    def transform(self, X):
        K = self.base_kernel.transform(X)
        return self.scale * K


class ExpKernel(Kernel):
    def __init__(self, base_kernel):
        self.base_kernel = base_kernel

    def fit_transform(self, X):
        K = self.base_kernel.fit_transform(X)
        return np.exp(K)
        
    def transform(self, X):
        K = self.base_kernel.transform(X)
        return np.exp(K)


class SumKernel(Kernel):
    def __init__(self, base_kernel1, base_kernel2):
        self.base_kernel1 = base_kernel1
        self.base_kernel2 = base_kernel2

    def fit_transform(self, X):
        K1 = self.base_kernel1.fit_transform(X)
        K2 = self.base_kernel2.fit_transform(X)
        return K1 + K2
        
    def transform(self, X):
        K1 = self.base_kernel1.transform(X)
        K2 = self.base_kernel2.transform(X)
        return K1 + K2

my_kernels = [ExpKernel(GraphKernel(kernel=[{"name": "shortest_path"}], normalize=True)),
              ScaledKernel(ExpKernel(GraphKernel(kernel=[{"name": "core_framework"}], normalize=True)), scale = 4.0),
              SumKernel(GraphKernel(kernel=[{"name": "shortest_path"}], normalize=True), ExpKernel(GraphKernel(kernel=[{"name": "weisfeiler_lehman"}], normalize=True)))
]

In [12]:
for dataset_name in graph_dataset_names:
    
    print("--------------------------------")
    print("dataset: ", dataset_name)
    print("--------------------------------")
    
    loaded = np.load(f'datasets/{dataset_name}.npz', allow_pickle=True)
    A_train = list(loaded['tr_adj'])
    X_train = loaded['tr_feat']
    y_train = np.argmax(loaded['tr_class'], axis=-1)
    A_test = list(loaded['te_adj'])
    X_test = loaded['te_feat']
    y_test = np.argmax(loaded['te_class'], axis=-1)

    X_train = np.nan_to_num(X_train)
    X_test = np.nan_to_num(X_test)
    
    G_tr, G_te = [], []
    for a, x in zip(A_train, X_train):
        G = nx.from_scipy_sparse_array(a)
        nx.set_node_attributes(G, dict(enumerate(map(tuple, x))), 'features')
        G_tr.append(G)
    for a, x in zip(A_test, X_test):
        G = nx.from_scipy_sparse_array(a)
        nx.set_node_attributes(G, dict(enumerate(map(tuple, x))), 'features')
        G_te.append(G)
    
    G_train = [g for g in graph_from_networkx(G_tr, node_labels_tag='features')]
    G_test = [g for g in graph_from_networkx(G_te, node_labels_tag='features')]
    
    
    
    for gk in my_kernels:
        start = time()
        K_train = np.nan_to_num(gk.fit_transform(G_train))
        K_test = np.nan_to_num(gk.transform(G_test))
        
        svm_clf = svm.SVC(kernel='precomputed', C=1)
        svm_clf.fit(K_train, y_train)
        svm_y_pred = svm_clf.predict(K_test)
        svm_acc = accuracy_score(y_test, svm_y_pred)
    
        ridge_y_pred = ridge_clf(K_train, y_train, K_test)
        ridge_acc = accuracy_score(y_test, ridge_y_pred)
    
        end = time()
        print(f"{gk} -- SVM Accuracy: {round(svm_acc*100, 2)}% | Ridge Accuracy: {round(ridge_acc*100, 2)}% | Time: {round(end - start, 2)}s")

    print("--------------------------------\n\n\n")

--------------------------------
dataset:  hard_small
--------------------------------
ExpKernel(base_kernel=GraphKernel(kernel=[{'name': 'shortest_path'}],
                                  normalize=True)) -- SVM Accuracy: 73.08% | Ridge Accuracy: 46.15% | Time: 2.72s
ScaledKernel(base_kernel=ExpKernel(base_kernel=GraphKernel(kernel=[{'name': 'core_framework'}],
                                                           normalize=True)),
             scale=4.0) -- SVM Accuracy: 76.92% | Ridge Accuracy: 46.15% | Time: 13.18s
SumKernel(base_kernel1=GraphKernel(kernel=[{'name': 'shortest_path'}],
                                   normalize=True),
          base_kernel2=ExpKernel(base_kernel=GraphKernel(kernel=[{'name': 'weisfeiler_lehman'}],
                                                         normalize=True))) -- SVM Accuracy: 65.38% | Ridge Accuracy: 42.31% | Time: 3.1s
--------------------------------





Получилось побить базовые ядра с помощью 4 * экспоненту от core_framework (SVM Accuracy: 76.92%)

# Задание 3.

Подберите несколько датасетов с большим количеством предикторов (>20) из UCI репозитория или любого другого (классификация и/или регрессия)). Реализуйте gaussian process (можно взять готовый но скорее всего его все равно придется модфицировать) для подбора гиперпараметров random forest (или какого либо бустинга) (смотрите лекцию 3). Сравните скорость с random search (попытайтесь презойти).

**Дополнительно**: поэксперементируйте с критериями подбора следующей точки (лекция 3) и ядрами, а также с самими критериями качеста (есть ли отличие в эффективности gaussian process для разных критериев)

P. S.: в этом задании оцениваться будет прежде всего правильность реализации, а не скорость, т.к. сделать быстрее рандома (в виду требуемых доп вычислений) может быть сложно и это зависит от датасета, но нужно попытаться реализовать как можно эффективнее.

In [13]:
import numpy as np
from time import time
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical
from scipy.stats import randint

In [15]:
def execute_optimization(features, labels, method_name, optimizer):
    start_time = time()
    optimizer.fit(features, labels)
    duration = time() - start_time
    best_score = optimizer.best_score_
    return {"best_score": best_score, "time": duration}

def bayes_optimization(features, labels, model, cv_splitter, iterations):
    bayes_optimizer = BayesSearchCV(
        estimator=model,
        search_spaces={
            "n_estimators": Integer(10, 200),
            "max_depth": Integer(1, 20),
            "min_samples_split": Integer(2, 20),
            "min_samples_leaf": Integer(1, 20),
            "bootstrap": Categorical([True, False]),
        },
        n_iter=iterations,
        n_jobs=-1,
        cv=cv_splitter,
        return_train_score=False,
        optimizer_kwargs={"base_estimator": "GP"},
    )
    return execute_optimization(features, labels, "BayesSearchCV (GP)", bayes_optimizer)

def random_search_optimization(features, labels, model, cv_splitter, iterations):
    param_distribution = {
        "n_estimators": randint(10, 200),
        "max_depth": randint(1, 20),
        "min_samples_split": randint(2, 20),
        "min_samples_leaf": randint(1, 20),
        "bootstrap": [True, False],
    }

    random_optimizer = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distribution,
        n_iter=iterations,
        n_jobs=-1,
        cv=cv_splitter,
        return_train_score=False,
        scoring="accuracy",
    )
    return execute_optimization(features, labels, "RandomizedSearchCV", random_optimizer)



dataset_collection = {}
dataset_list = ["spambase", "steel-plates-fault", "one-hundred-plants-shape", "scene"]

for dataset in dataset_list:
    data = fetch_openml(name=dataset, as_frame=False)
    features, labels = data.data, data.target.astype(int)
    dataset_collection[dataset] = (features, labels)

cv_splitter = StratifiedKFold(n_splits=5, random_state=22, shuffle=True)
base_model = RandomForestClassifier(random_state=42)
iteration_count = 40

results = {}
for dataset_name, (features, labels) in dataset_collection.items():
    print(f"\nProcessing dataset: {dataset_name}")
    results[dataset_name] = {}
    results[dataset_name]["BayesSearchCV"] = bayes_optimization(features, labels, base_model, cv_splitter, iteration_count)
    results[dataset_name]["RandomizedSearchCV"] = random_search_optimization(features, labels, base_model, cv_splitter, iteration_count)

    print(f"{'Method':<20}{'Best Score':<15}{'Time (s)':<10}")
    print("-" * 45)
    for method_name, metrics in results[dataset_name].items():
        print(f"{method_name:<20}{metrics['best_score']:<15.4f}{metrics['time']:<10.2f}")




Processing dataset: spambase
Method              Best Score     Time (s)  
---------------------------------------------
BayesSearchCV       0.9520         79.07     
RandomizedSearchCV  0.9446         16.82     

Processing dataset: steel-plates-fault
Method              Best Score     Time (s)  
---------------------------------------------
BayesSearchCV       0.9964         73.16     
RandomizedSearchCV  0.9948         13.63     

Processing dataset: one-hundred-plants-shape
Method              Best Score     Time (s)  
---------------------------------------------
BayesSearchCV       0.6400         201.40    
RandomizedSearchCV  0.6138         68.73     

Processing dataset: scene
Method              Best Score     Time (s)  
---------------------------------------------
BayesSearchCV       0.9244         240.19    
RandomizedSearchCV  0.9206         68.96     


Получили результаты с помощью gaussian process немного лучше чем через random search, но затраченное время сильно выше
